In [3]:
# App_Rose_Diagram_Jupyter.ipynb (versión Jupyter con ipywidgets)

import pandas as pd
import matplotlib.pyplot as plt
from windrose import WindroseAxes
import itertools
import ipywidgets as widgets
from IPython.display import display, clear_output

In [4]:
# --- Cargar datos ---
RUTA_CSV = '/workspaces/ml-bootcamp-labs/data/raw/DATOS_DIAGRAMA_ROSA_UNIFICADO.csv'
df = pd.read_csv(RUTA_CSV, sep=';')

In [5]:
# --- Rangos por pozo ---
rangos_pozo = (
    df.groupby('wellName', as_index=False)['TDEP-ft']
      .agg(prof_min='min', prof_max='max')
      .sort_values('wellName')
)
print("📊 Rangos por pozo (ft):")
display(rangos_pozo)

if 'Type' in df.columns:
    rangos_pozo_tipo = (
        df.groupby(['wellName', 'Type'], as_index=False)['TDEP-ft']
          .agg(prof_min='min', prof_max='max')
          .sort_values(['wellName', 'Type'])
    )
    print("📊 Rangos por pozo y tipo (ft):")
    display(rangos_pozo_tipo)

📊 Rangos por pozo (ft):


,wellName,prof_min,prof_max
0,CARDALES_14,2740.898077,8276.299622
1,CARDALES_1N,5597.881041,8698.683587
2,GALAN-124,4528.061124,7795.514706
3,GALAN_136,2431.731825,6482.286341
4,GALA_16,2727.405380,7786.545728
5,LLANITO 123,5297.177142,7997.569448
6,LLANITO 133,5485.562511,7572.335263
7,LLANITO-108,5192.940662,7619.300339
8,LLANITO_112,5089.772243,7700.419096
9,LLANITO_116,5239.289875,7822.306505


📊 Rangos por pozo y tipo (ft):


,wellName,Type,prof_min,prof_max
0,CARDALES_14,Bedding,2741.016849,8276.299622
1,CARDALES_14,Conductive Fracture,4194.333036,5748.264712
2,CARDALES_14,Cross bedding,2744.073789,8251.729146
3,CARDALES_14,Erosional surface,2912.137543,7563.472924
4,CARDALES_14,Fault,4320.829361,7247.506528
...,...,...,...,...
123,llanito_107,Fault,3357.914999,7858.440558
124,llanito_107,Open Fracture,6073.208218,7350.858627
125,llanito_107,Partially Open Fracture,2596.447905,7999.083225
126,llanito_107,Resistive Fracture,2602.394967,7892.466044


In [6]:
# --- Widgets de selección ---
pozos = sorted(df['wellName'].dropna().unique())
pozo_sel = widgets.Dropdown(options=pozos, description="Pozo:")

def update_tipo_options(*args):
    tipos = sorted(df.loc[df['wellName'] == pozo_sel.value, 'Type'].dropna().unique())
    tipo_sel.options = tipos

tipo_sel = widgets.Dropdown(description="Tipo:")
pozo_sel.observe(update_tipo_options, names='value')
update_tipo_options()

In [7]:
# Entradas de profundidad (valores por defecto luego de elegir pozo+tipo)
prof_min_input = widgets.FloatText(description="Prof Min (ft):")
prof_max_input = widgets.FloatText(description="Prof Max (ft):")

def update_prof_range(*args):
    df_tmp = df[(df['wellName'] == pozo_sel.value) & (df['Type'] == tipo_sel.value)]
    if not df_tmp.empty:
        prof_min = float(df_tmp['TDEP-ft'].min())
        prof_max = float(df_tmp['TDEP-ft'].max())
        prof_min_input.value = prof_min
        prof_max_input.value = prof_max

tipo_sel.observe(update_prof_range, names='value')
pozo_sel.observe(update_prof_range, names='value')
update_prof_range()

In [8]:
# Número de bins
bins_slider = widgets.IntSlider(value=8, min=4, max=36, description="Bins:")

In [9]:
# Paletas de colores
paletas = {
    'Clásico Geológico': ['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ffa600'],
    'Elegante': ['#264653', '#2a9d8f', '#e9c46a', '#f4a261', '#e76f51'],
    'Marino/Terrestre': ['#00429d', '#4771b2', '#73a2c6', '#a5d5d8', '#f6f5f5']
}
paleta_sel = widgets.Dropdown(options=list(paletas.keys()), value='Clásico Geológico', description="Paleta:")


In [10]:
# Modo
modo_sel = widgets.RadioButtons(
    options=["Orientaciones puras (solo azimut)", "Orientaciones + clasificación por dip"],
    description="Modo:"
)

In [11]:
# Botón para generar
btn = widgets.Button(description="Generar diagrama", button_style="success")

In [15]:
# --- Función para graficar ---
def generar_diagrama(b):
    clear_output(wait=True)
    display(ui)  # vuelve a mostrar UI
    
    df_tmp = df[(df['wellName'] == pozo_sel.value) & (df['Type'] == tipo_sel.value)]
    if df_tmp.empty:
        print("⚠️ No hay datos para esta selección.")
        return

    # Filtrado por profundidades
    df_f = df_tmp[
        (df_tmp['TDEP-ft'] >= prof_min_input.value) &
        (df_tmp['TDEP-ft'] <= prof_max_input.value)
    ]

    if df_f.empty:
        print("⚠️ No hay datos en el rango de profundidad.")
        return

    # ✅ Aquí va la selección de colores SIN indentación extra
    colores = paletas[paleta_sel.value]
    colores_final = list(itertools.islice(itertools.cycle(colores), bins_slider.value))

In [19]:
def generar_diagrama(b):
    clear_output(wait=True)
    display(ui)  # vuelve a mostrar la UI
    
    df_tmp = df[(df['wellName'] == pozo_sel.value) & (df['Type'] == tipo_sel.value)]
    if df_tmp.empty:
        print("⚠️ No hay datos para esta selección.")
        return

    # Filtrado por profundidades
    df_f = df_tmp[
        (df_tmp['TDEP-ft'] >= prof_min_input.value) &
        (df_tmp['TDEP-ft'] <= prof_max_input.value)
    ]

    if df_f.empty:
        print("⚠️ No hay datos en el rango de profundidad.")
        return

    # Paleta de colores
    colores = paletas[paleta_sel.value]
    colores_final = list(itertools.islice(itertools.cycle(colores), bins_slider.value))

    # Crear figura
    fig = plt.figure(figsize=(7, 7))
    ax = WindroseAxes.from_ax(fig=fig)

    if modo_radio.value == "Orientaciones puras (solo azimut)":
        ax.bar(
            df_f['Azimuth-dega'],
            [1]*len(df_f),
            bins=bins_slider.value,
            normed=True,
            opening=0.8,
            edgecolor='black',
            colors=colores_final
        )
        ax.set_legend(title="Frecuencia (%)", loc='lower right', bbox_to_anchor=(1.2, 0.1))
        plt.title(
            f"Diagrama de Rosa (Orientaciones) - {pozo_sel.value} - {tipo_sel.value}\n"
            f"{prof_min_input.value:.2f}–{prof_max_input.value:.2f} ft",
            pad=30
        )
    else:
        ax.bar(
            df_f['Azimuth-dega'],
            df_f['Dip_dega'],
            bins=bins_slider.value,
            normed=True,
            opening=0.8,
            edgecolor='black',
            colors=colores_final
        )
        ax.set_legend(title="Frecuencia (%)", loc='lower right', bbox_to_anchor=(1.2, 0.1))
        plt.title(
            f"Diagrama de Rosa (Orientaciones + Dip) - {pozo_sel.value} - {tipo_sel.value}\n"
            f"{prof_min_input.value:.2f}–{prof_max_input.value:.2f} ft",
            pad=30
        )

    plt.show()
